In [1]:
import pandas as pd

In [2]:
import os
data_folder = './DATA'

In [3]:
nomination_file_names = [
    'best_picture_academy_awards'
]

In [4]:
import sys
sys.path.append('../../..')
import academy_award_predictor_constants as c

In [5]:
def get_data_df(nomination_file_name):
    filename = "{0}_{1}{2}".format(nomination_file_name, c.NOMINATION_FILE_DATA, c.NOMINATION_FILE_PREFIX)
    df_data = pd.read_csv(os.path.join(data_folder, filename), index_col=0)
    df_data.fillna(0, inplace=True)
    return df_data

In [6]:
def get_predict_df(nomination_file_name):
    filename = "{0}_{1}{2}".format(nomination_file_name, c.NOMINATION_FILE_PREDICT, c.NOMINATION_FILE_PREFIX)
    df_predict = pd.read_csv(os.path.join(data_folder, filename), index_col=0)
    df_predict.fillna(0, inplace=True)
    return df_predict

In [7]:
def get_x_values(df):
    cols = ['Nomination_Film_Title', ]
    if 'Nomination_Is_Winner' in df.columns:
        cols.append('Nomination_Is_Winner')
    return df.drop(columns=cols).values

def get_y_values(df):
    return df['Nomination_Is_Winner'].values

def get_titles(df):
    return df['Nomination_Film_Title'].values

In [8]:
from collections import Counter

def get_y_distribution(y):
    counter = Counter(y)
    print("win: {0}  \tloose: {1} \t{2:.1f}%".format(counter[1], counter[0], counter[1]/(counter[1] + counter[0])*100))

In [9]:
def get_data_values(nomination_file_name):
    df_data = get_data_df(nomination_file_name)
    
    X = get_x_values(df_data)
    Y = get_y_values(df_data)
    print("Distribution of full data:")
    get_y_distribution(Y)
    return X, Y

def get_predict_values(nomination_file_name):
    df_predict = get_predict_df(nomination_file_name)
    
    X_predict = get_x_values(df_predict)
    X_predict_titles = get_titles(df_predict)
    return X_predict, X_predict_titles

In [10]:
from sklearn.model_selection import train_test_split

def get_data_split(nomination_file_name):
    X, Y = get_data_values(nomination_file_name)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    print("Distribution of training data")
    get_y_distribution(Y_train)
    print("Distribution of test data")
    get_y_distribution(Y_test)
    return X_train, X_test, Y_train, Y_test

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
import timeit

def predict_for_clf(clf_best, x_row):
    prediction = clf_best.predict_proba(x_row.reshape(1, -1))
    return prediction[0][1]

def gridsearch_for_clf(nomination_file_name, clf, params, name):
    print(name)
    X_train, X_test, Y_train, Y_test = get_data_split(nomination_file_name)
    grid_search = GridSearchCV(clf, params, scoring='roc_auc', n_jobs=1, cv=5)

    start_time = timeit.default_timer()

    grid_search.fit(X_train, Y_train)

    print("--- %0.3fs seconds ---" % (timeit.default_timer() - start_time))

    print(grid_search.best_params_)
    print('The accuracy after grid search is {0:.1f}%'.format(grid_search.best_score_*100))

    clf_best = grid_search.best_estimator_

    accuracy = clf_best.score(X_test, Y_test)
    print('The accuracy on the test set is {0:.1f}%'.format(accuracy*100))
    print("-----------------------------------------")
    X_predict, X_predict_titles = get_predict_values(nomination_file_name)
    print("Predicting the nominees")
    results = [{
        'prediction': predict_for_clf(clf_best, x_row),
        'title': X_predict_titles[idx]
    } for idx, x_row in enumerate(X_predict)]
    for res in sorted(results, key=lambda res: res['prediction'], reverse=True):
        print('{0:.1f}%\t:  {1}'.format(res['prediction'] * 100, res['title']))
    print("====================================================================")

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

classifiers = [{
    'name': 'Multinomial Naive Bayes',
    'clf': MultinomialNB(),
    'params': {
        'fit_prior': (True, False),
        'alpha': (0.5, 1.0, 2.0, 4.0),
    }
},{
    'name': 'Random Forest',
    'clf': RandomForestClassifier(criterion='gini', n_jobs=-1),
    'params': {
        'n_estimators': (100, 200, 300),
        'min_samples_split': (20, 30, 40),
        'max_features': ('sqrt', 'log2', None)
    }
}]

def gridsearch_for_file(nomination_file_name):
    for classifier in classifiers:
        gridsearch_for_clf(nomination_file_name, classifier['clf'], classifier['params'], classifier['name'])

In [13]:
for nomination_file_name in nomination_file_names:
    gridsearch_for_file(nomination_file_name)

Multinomial Naive Bayes
Distribution of full data:
win: 92  	loose: 465 	16.5%
Distribution of training data
win: 74  	loose: 371 	16.6%
Distribution of test data
win: 18  	loose: 94 	16.1%
--- 0.141s seconds ---
{'alpha': 0.5, 'fit_prior': True}
The accuracy after grid search is 64.8%
The accuracy on the test set is 64.3%
-----------------------------------------
Predicting the nominees

C:\Users\Rik\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)



100.0%	:  Gisaengchung  
95.5%	:  Marriage Story  
6.6%	:  Ford v Ferrari  
0.7%	:  Jojo Rabbit  
0.1%	:  1917  
0.0%	:  Joker  
0.0%	:  Little Women  
0.0%	:  Once Upon a Time ...in Hollywood  
0.0%	:  The Irishman  
Random Forest
Distribution of full data:
win: 92  	loose: 465 	16.5%
Distribution of training data
win: 74  	loose: 371 	16.6%
Distribution of test data
win: 18  	loose: 94 	16.1%
--- 42.959s seconds ---
{'max_features': 'sqrt', 'min_samples_split': 30, 'n_estimators': 100}
The accuracy after grid search is 80.3%
The accuracy on the test set is 83.9%
-----------------------------------------
Predicting the nominees
37.0%	:  Joker  
36.9%	:  The Irishman  
30.7%	:  1917  
30.6%	:  Once Upon a Time ...in Hollywood  
16.0%	:  Gisaengchung  
15.6%	:  Marriage Story  
12.0%	:  Little Women  
10.8%	:  Ford v Ferrari  
6.1%	:  Jojo Rabbit  
